In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

/content/drive/.shortcut-targets-by-id/1bqqm9-0FMOqiqyH_NIQBh-lJ36pWa7aX/Masters_Project/Datasets/OASIS3


In [28]:
!ls

 cifar-10-batches-py	  downloaded_data   t1w_non_ad_jpgs
 cifar-10-python.tar.gz   fa20		    t1w_non_ad_patients
 data			  oasis-scripts    'Usage Instructions.gdoc'


In [4]:
!pip install -r ../../Code/ganomaly/requirements.txt
#!pip install -r ./ganomaly/requirements.txt

In [5]:
!pip install mkl-fft

  Using cached https://files.pythonhosted.org/packages/e1/de/fe883f978594e49075fe57161a4d0327687f57f47152ebe505fb8e6f276c/mkl_fft-1.2.0-10-cp37-cp37m-manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/65/08/f6efbd186b59eb84b132cfc30effa0597e60489589ce9fc2d3e3035f75e5/dpcpp_cpp_rt-2021.1.2-py2.py3-none-manylinux1_x86_64.whl
     |████████████████████████████████| 14.8MB 533kB/s 
     |████████████████████████████████| 153.4MB 79kB/s 
     |████████████████████████████████| 30.6MB 1.5MB/s 
     |████████████████████████████████| 3.6MB 33.0MB/s 
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.12.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.33.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.1.0 whi

In [6]:
!pip install Pillow

# imports

In [7]:
import os
import glob
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


In [8]:
from PIL import Image
#import cv2
import os.path
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [9]:
print(torch.__version__)

1.2.0


In [10]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [22]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])# https://github.com/yunjey/pytorch-tutorial/issues/161
        
    ]
)

In [23]:
train_image_number = 140 # Image slice from MRI scans to use for training data

base_folder = '/pyt_oasis3'
non_ad_jpg_folder = './t1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = './data/t1w_ad_jpgs/'

class Oasis3_train_normal(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3_train_normal, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set
        
        self.data = []
        self.targets = []
        
        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.data.append(resized)
                self.targets.extend(str(0)) # based on the jpg_folder in for loop
        self.data = np.vstack(self.data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.data = self.data.transpose((0, 1, 2))  # convert to HWC

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (img, target)

In [24]:
# Issue is i need to make the data transformable inside __getitem__
a=Oasis3_train_normal(base_folder, False, transform=transform)

In [25]:
# Shape : (256, 256)
a.data[0].shape

(256, 256)

In [26]:
# The class should be index into which invokes the __getitem__ call
print(a[0][0])
print(a[0][1])

tensor([[[-0.9922, -0.9608, -0.9922,  ..., -0.9686, -0.9843, -1.0000],
         [-0.9843, -0.9608, -0.9922,  ..., -0.9686, -0.9686, -1.0000],
         [-0.9843, -0.9686, -0.9922,  ..., -0.9686, -0.9451, -1.0000],
         ...,
         [-0.9451, -0.9765, -0.9686,  ..., -0.9843, -0.9765, -1.0000],
         [-0.9529, -0.9843, -0.9686,  ..., -0.9686, -0.9686, -1.0000],
         [-0.9608, -0.9922, -0.9686,  ..., -0.9608, -0.9686, -1.0000]]])
0
